In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from IPython.core.display import display, HTML
from cross_validation import *
from build_polynomial import *
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))


## Load the training data into feature matrix, class labels, and event ids:

In [3]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
#y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
#print(y.shape, tX.shape, ids.shape)

In [3]:
import pandas as pd
df = pd.read_csv("../data/train.csv")
df.tail()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
249995,349995,b,-999.000,71.989,36.548,5.042,-999.0,-999.0,-999.0,1.392,...,2.859,144.665,0,-999.000,-999.0,-999.000,-999.0,-999.0,-999.0,0.000
249996,349996,b,-999.000,58.179,68.083,22.439,-999.0,-999.0,-999.0,2.585,...,-0.867,80.408,0,-999.000,-999.0,-999.000,-999.0,-999.0,-999.0,0.000
249997,349997,s,105.457,60.526,75.839,39.757,-999.0,-999.0,-999.0,2.390,...,-2.890,198.907,1,41.992,1.8,-0.166,-999.0,-999.0,-999.0,41.992
249998,349998,b,94.951,19.362,68.812,13.504,-999.0,-999.0,-999.0,3.365,...,0.811,112.718,0,-999.000,-999.0,-999.000,-999.0,-999.0,-999.0,0.000
249999,349999,b,-999.000,72.756,70.831,7.479,-999.0,-999.0,-999.0,2.025,...,-1.596,99.405,0,-999.000,-999.0,-999.000,-999.0,-999.0,-999.0,0.000


In [37]:
y0, tX0, id0 = load_data_jet_number(DATA_TRAIN_PATH, 0, sub_sample=True)
y1, tX1, id1 = load_data_jet_number(DATA_TRAIN_PATH, 1, sub_sample=True)
y2, tX2, id2 = load_data_jet_number(DATA_TRAIN_PATH, 2, sub_sample=True)
y3, tX3, id3 = load_data_jet_number(DATA_TRAIN_PATH, 3, sub_sample=True)
tX0_standardized, tX0_mean, tX0_std= standardize(tX0)
tX1_standardized, tX1_mean, tX1_std= standardize(tX1)
tX2_standardized, tX2_mean, tX2_std= standardize(tX2)
tX3_standardized, tX3_mean, tX3_std= standardize(tX3)

In [38]:
print(y0.shape, tX0.shape, id0.shape)

(1999,) (1999, 18) (1999,)


## Do your thing crazy machine learning thing here :) ...

In [39]:
tX0_standardized = np.c_[np.ones((y0.shape[0], 1)), tX0_standardized] #VERY IMPORTANT
y0=y0.reshape(-1,1)
y0=(y0+1)/2
tX1_standardized = np.c_[np.ones((y1.shape[0], 1)), tX1_standardized] #VERY IMPORTANT
y1=y1.reshape(-1,1)
y1=(y1+1)/2
tX2_standardized = np.c_[np.ones((y2.shape[0], 1)), tX2_standardized] #VERY IMPORTANT
y2=y2.reshape(-1,1)
y2=(y2+1)/2
tX3_standardized = np.c_[np.ones((y3.shape[0], 1)), tX3_standardized] #VERY IMPORTANT
y3=y3.reshape(-1,1)
y3=(y3+1)/2

In [40]:
print(tX0_standardized.shape)

(1999, 19)


In [55]:
[w,loss]=reg_logistic_regression(y0, tX0_standardized, 0.001,np.ones((tX0_standardized.shape[1], 1)), 10000, 0.0001)

Current iteration = 0, loss = [[1.78286643]] Current iteration = 0, loss = [[1.78250965]] Current iteration = 0, loss = [[1.78230085]] Current iteration = 0, loss = [[1.78209666]] Current iteration = 2000, loss = [[0.64380394]] rate = [[1.6848015e-05]]
Current iteration = 2000, loss = [[0.64373481]] rate = [[6.72778714e-05]]
Current iteration = 2000, loss = [[0.64368807]] rate = [[6.91380097e-05]]
Current iteration = 2000, loss = [[0.64361968]] rate = [[4.67317794e-05]]
Current iteration = 4000, loss = [[0.56208761]] rate = [[-7.82253763e-07]]
Current iteration = 4000, loss = [[0.56207027]] rate = [[4.55162546e-06]]
Current iteration = 4000, loss = [[0.56206276]] rate = [[1.73426308e-05]]
Current iteration = 4000, loss = [[0.56206517]] rate = [[7.50167423e-06]]
Current iteration = 6000, loss = [[0.55290379]] rate = [[1.93488194e-06]]
Current iteration = 6000, loss = [[0.55290048]] rate = [[3.60994299e-06]]
Current iteration = 6000, loss = [[0.55289864]] rate = [[3.30544028e-06]]
Curren

In [ ]:
#lambda_=0.0000001
k_fold=4
seed=5
degree=5 #will -1 as range exclude last elem
lower_lambda=-8
upper_lambda=0

weights_0, loss_0, deg0 = cross_validation_best_weight(y0, tX0_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet0")
weights_1, loss_1, deg1 = cross_validation_best_weight(y1, tX1_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet1")
weights_2, loss_2, deg2 = cross_validation_best_weight(y2, tX2_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet2")
weights_3, loss_3, deg3 = cross_validation_best_weight(y3, tX3_standardized, k_fold, degree, seed, lower_lambda, upper_lambda, "jet3")

DEGREE = 1
Lambda = 1e-08
K-Fold =  0
Current iteration = 0, loss = [[3.18843101]] Current iteration = 0, loss = [[3.13169388]] Current iteration = 0, loss = [[3.06310833]] Current iteration = 0, loss = [[3.01563406]] Current iteration = 2000, loss = [[0.56534083]] rate = [[-0.00021739]]
Current iteration = 2000, loss = [[0.56591219]] rate = [[0.00191321]]
Current iteration = 2000, loss = [[0.56540905]] rate = [[-0.00057136]]
Current iteration = 2000, loss = [[0.56592141]] rate = [[0.00050314]]
Current iteration = 4000, loss = [[0.56568412]] rate = [[0.00022378]]
Current iteration = 4000, loss = [[0.56657589]] rate = [[-0.00022155]]
Current iteration = 4000, loss = [[0.56592892]] rate = [[-0.00089176]]
Current iteration = 4000, loss = [[0.56668542]] rate = [[0.00064697]]
Current iteration = 6000, loss = [[0.5733119]] rate = [[0.00066132]]
Current iteration = 6000, loss = [[0.57299829]] rate = [[-0.00505277]]
Current iteration = 6000, loss = [[0.57299118]] rate = [[0.00031361]]
Current 

Best last loss = 0.5669060104582714 after 9999 iterations 
K-Fold =  2
Current iteration = 0, loss = [[3.25200029]] Current iteration = 0, loss = [[3.13853906]] Current iteration = 0, loss = [[3.0863931]] Current iteration = 0, loss = [[3.06299085]] Current iteration = 2000, loss = [[0.5608529]] rate = [[-0.00152605]]
Current iteration = 2000, loss = [[0.56075896]] rate = [[0.00111489]]
Current iteration = 2000, loss = [[0.5607774]] rate = [[9.39459841e-05]]
Current iteration = 2000, loss = [[0.56196683]] rate = [[-1.84401211e-05]]
Current iteration = 4000, loss = [[0.56853633]] rate = [[0.00290544]]
Current iteration = 4000, loss = [[0.56845595]] rate = [[-0.00013192]]
Current iteration = 4000, loss = [[0.56952697]] rate = [[8.03774866e-05]]
Current iteration = 4000, loss = [[0.5696884]] rate = [[-0.00107102]]
Current iteration = 6000, loss = [[0.5590896]] rate = [[0.00418072]]
Current iteration = 6000, loss = [[0.55824134]] rate = [[-0.00013341]]
Current iteration = 6000, loss = [[0.

Best last loss = 0.5781539089352385 after 9999 iterations 
Lambda = 4.641588833612773e-06
K-Fold =  0
Current iteration = 0, loss = [[3.18843101]] Current iteration = 0, loss = [[3.09872244]] Current iteration = 0, loss = [[3.09580059]] Current iteration = 0, loss = [[3.07016966]] Current iteration = 2000, loss = [[0.57284834]] rate = [[-0.00296665]]
Current iteration = 2000, loss = [[0.57434511]] rate = [[0.00180594]]
Current iteration = 2000, loss = [[0.57399084]] rate = [[-0.00149678]]
Current iteration = 2000, loss = [[0.57244202]] rate = [[0.00035427]]
Current iteration = 4000, loss = [[0.56705037]] rate = [[0.00178948]]
Current iteration = 4000, loss = [[0.56686649]] rate = [[-0.00049891]]
Current iteration = 4000, loss = [[0.56633059]] rate = [[0.00018388]]
Current iteration = 4000, loss = [[0.56677249]] rate = [[0.0005359]]
Current iteration = 6000, loss = [[0.56851334]] rate = [[0.00030881]]
Current iteration = 6000, loss = [[0.56862954]] rate = [[0.00023666]]
Current iteratio

Best last loss = 0.5647487495176888 after 9999 iterations 
K-Fold =  2
Current iteration = 0, loss = [[3.25200029]] Current iteration = 0, loss = [[3.19314305]] Current iteration = 0, loss = [[3.09699463]] Current iteration = 0, loss = [[3.0060167]] Current iteration = 2000, loss = [[0.56542406]] rate = [[0.00042227]]
Current iteration = 2000, loss = [[0.56408846]] rate = [[5.14361254e-05]]
Current iteration = 2000, loss = [[0.56393636]] rate = [[0.0013356]]
Current iteration = 2000, loss = [[0.56512299]] rate = [[0.0001521]]
Current iteration = 4000, loss = [[0.57513515]] rate = [[-0.00039399]]
Current iteration = 4000, loss = [[0.57651616]] rate = [[-0.00125423]]
Current iteration = 4000, loss = [[0.5756311]] rate = [[-0.00138101]]
Current iteration = 4000, loss = [[0.57432118]] rate = [[0.00088507]]
Current iteration = 6000, loss = [[0.56313018]] rate = [[-0.00050908]]
Current iteration = 6000, loss = [[0.5635354]] rate = [[-0.00086994]]
Current iteration = 6000, loss = [[0.56346038

Best last loss = 0.5827624769848389 after 9999 iterations 
Lambda = 0.002154434690031882
K-Fold =  0
Current iteration = 0, loss = [[3.18843101]] Current iteration = 0, loss = [[3.11768532]] Current iteration = 0, loss = [[3.09325751]] Current iteration = 0, loss = [[2.99792919]] Current iteration = 2000, loss = [[0.57243313]] rate = [[-0.00065303]]
Current iteration = 2000, loss = [[0.5783188]] rate = [[0.00087276]]
Current iteration = 2000, loss = [[0.57547034]] rate = [[-0.00588568]]
Current iteration = 2000, loss = [[0.57700789]] rate = [[0.00284846]]
Current iteration = 4000, loss = [[0.57271181]] rate = [[0.00101574]]
Current iteration = 4000, loss = [[0.57247071]] rate = [[0.00010944]]
Current iteration = 4000, loss = [[0.57262797]] rate = [[0.0002411]]
Current iteration = 4000, loss = [[0.57245007]] rate = [[-0.00015726]]
Current iteration = 6000, loss = [[0.57582819]] rate = [[-0.00106966]]
Current iteration = 6000, loss = [[0.57528497]] rate = [[-0.00164173]]
Current iteratio

Best last loss = 0.5664711836175566 after 9999 iterations 
K-Fold =  2
Current iteration = 0, loss = [[3.25200029]] Current iteration = 0, loss = [[3.21020846]] Current iteration = 0, loss = [[3.15796911]] Current iteration = 0, loss = [[3.13511834]] Current iteration = 2000, loss = [[0.56112896]] rate = [[-0.00040511]]
Current iteration = 2000, loss = [[0.5611719]] rate = [[-0.00012387]]
Current iteration = 2000, loss = [[0.56162212]] rate = [[-4.29409856e-05]]
Current iteration = 2000, loss = [[0.56223862]] rate = [[-0.00045022]]
Current iteration = 4000, loss = [[0.56838821]] rate = [[1.55730612e-05]]
Current iteration = 4000, loss = [[0.56824286]] rate = [[-0.00113036]]
Current iteration = 4000, loss = [[0.56758954]] rate = [[0.00014535]]
Current iteration = 4000, loss = [[0.56878276]] rate = [[0.00065333]]
Current iteration = 6000, loss = [[0.56150394]] rate = [[0.00166908]]
Current iteration = 6000, loss = [[0.56230586]] rate = [[0.00163137]]
Current iteration = 6000, loss = [[0.

Best last loss = 0.5651857574078452 after 9999 iterations 
Lambda = 1.0
K-Fold =  0
Current iteration = 0, loss = [[3.18843101]] Current iteration = 0, loss = [[3.08278342]] Current iteration = 0, loss = [[3.05168305]] Current iteration = 0, loss = [[2.97364011]] Current iteration = 2000, loss = [[0.57018097]] rate = [[-0.00077153]]
Current iteration = 2000, loss = [[0.57039412]] rate = [[0.00064665]]
Current iteration = 2000, loss = [[0.57224755]] rate = [[-0.00021314]]
Current iteration = 2000, loss = [[0.57138969]] rate = [[-0.00185343]]
Current iteration = 4000, loss = [[0.56532519]] rate = [[-0.00071007]]
Current iteration = 4000, loss = [[0.56734138]] rate = [[9.06816842e-05]]
Current iteration = 4000, loss = [[0.56582822]] rate = [[-0.00201618]]
Current iteration = 4000, loss = [[0.56515328]] rate = [[0.00151315]]
Current iteration = 6000, loss = [[0.56808767]] rate = [[-0.00048982]]
Current iteration = 6000, loss = [[0.56852409]] rate = [[-0.00023]]
Current iteration = 6000, lo

# Generate predictions and save ouput in csv format for submission:

In [12]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
print(tX_test.shape, ids_test.shape)

In [13]:
_, tX0_te, id0_te = load_data_jet_number("../data/test.csv", 0)
_, tX1_te, id1_te = load_data_jet_number("../data/test.csv", 1)
_, tX2_te, id2_te = load_data_jet_number("../data/test.csv", 2)
_, tX3_te, id3_te = load_data_jet_number("../data/test.csv", 3)
tX0_te_standardized, tX0_te_mean, tX0_te_std= standardize(tX0_te)
tX1_te_standardized, tX1_te_mean, tX1_te_std= standardize(tX1_te)
tX2_te_standardized, tX2_te_mean, tX2_te_std= standardize(tX2_te)
tX3_te_standardized, tX3_te_mean, tX3_te_std= standardize(tX3_te)

In [22]:
y0_pred = predict_labels(w, a)

[[-2.75157258]
 [-0.47500703]
 [ 0.42537955]
 [-2.62267781]
 [-1.39500056]
 [-2.35267385]
 [-0.88730474]
 [-2.43046776]
 [-2.83740092]
 [-1.8801182 ]
 [-1.59146235]
 [-0.01874744]
 [-2.03022476]
 [-1.95865182]
 [-2.82259573]
 [-1.42175303]
 [-1.69852775]
 [-3.07193772]
 [-0.17078513]
 [-1.51216748]
 [ 0.06946659]
 [ 0.57790712]
 [-3.00188763]
 [-3.1048203 ]
 [ 0.37691754]
 [-0.94225998]
 [-1.08140198]
 [-2.10000513]
 [-1.4457207 ]
 [ 1.37125505]
 [ 0.12446946]
 [-2.46042879]
 [-0.73530546]
 [-2.03875331]
 [-2.72798913]
 [-0.16043609]
 [-5.96100614]
 [-1.19087834]
 [-1.51151616]
 [-1.07751154]
 [ 1.46794467]
 [-2.96578797]
 [-2.98386863]
 [-0.29048986]
 [-0.95067566]
 [-0.82838131]
 [ 0.08114055]
 [-1.48294799]
 [ 0.1902982 ]
 [-1.76157185]
 [-4.03077058]
 [-1.80779363]
 [ 0.2749607 ]
 [-0.77980002]
 [-1.84391736]
 [-1.44034842]
 [-1.50158567]
 [ 0.62098695]
 [-3.2029887 ]
 [-6.17945759]
 [-2.28723705]
 [ 0.6227722 ]
 [-2.8986828 ]
 [-0.22515421]
 [-1.38458   ]
 [-3.22968022]
 [-0.61552

In [21]:
print(y0_pred[0:100])

[[-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [ 1.]
 [-1.]
 [-1.]
 [ 1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]
 [ 1.]
 [-1.]]


In [ ]:
OUTPUT_PATH = '../data/pred.csv' # TODO: fill in desired name of output file for submission
y0_pred = predict_labels(weights_0, build_poly(tX0_te_standardized,deg0))
y1_pred = predict_labels(weights_1, build_poly(tX1_te_standardized,deg1))
y2_pred = predict_labels(weights_2, build_poly(tX2_te_standardized,deg2))
y3_pred = predict_labels(weights_3, build_poly(tX3_te_standardized,deg3))

In [ ]:
y_pred_all=np.concatenate([y0_pred,y1_pred,y2_pred,y3_pred])
ids_pred_all=np.concatenate([id0_te,id1_te,id2_te,id3_te])
y_and_ids=list(zip(*sorted(zip(ids_pred_all, y_pred_all))))
ids_test=y_and_ids[0]
y_pred=y_and_ids[1]

In [ ]:
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)